In [157]:
import os
import subprocess
import numpy as np
import pandas as pd
import random
import mplcursors
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

In [158]:
def prt_point(p, r):
    angle = np.random.uniform(0, 2 * np.pi)
    radius = np.sqrt(np.random.uniform(0, 1)) * r
    dx = radius * np.cos(angle)
    dy = radius * np.sin(angle)
    point = (p[0]+dx, p[1]+dy)
    return point

def run_cpp_program():
    result = subprocess.run(['../build/MyGTSAM'], 
                            capture_output=True, text=True)

    if result.returncode != 0:
        print(f"Error running C++ program: {result.stderr}")
        return np.inf

    # print(f"C++ program output:\n{result.stdout}")

    return result.stdout.strip().split('\n')

In [159]:
content = pd.read_csv('initial_estimates_7.csv', header=None)
initialEst = content.values

cable_GT = initialEst[50:54,0].copy()
anchor_GT = initialEst[54:,:].copy()

# Perturbation Results
    Anchor 1: 0.4
    Anchor 2: 0.3
    Anchor 3: 0.55
    Anchor 4: 0.4
    
    All Together: 0.2

In [160]:
result = []
anchor_prt = 0.3
cable_prt = 0.0
file_path = 'perturb.csv'
file_exists = os.path.isfile(file_path)
for i in range(100):
    initialEst[50,:] = cable_GT[0] + random.uniform(-cable_prt, cable_prt)  # m0
    initialEst[51,:] = cable_GT[1] + random.uniform(-cable_prt, cable_prt)  # n0
    initialEst[52,:] = cable_GT[2] + random.uniform(-cable_prt, cable_prt)  # k0
    initialEst[53,:] = cable_GT[3] + random.uniform(-cable_prt, cable_prt)  # h0

    initialEst[54,:] = prt_point(anchor_GT[0],anchor_prt)    # Anchor 1
    initialEst[55,:] = prt_point(anchor_GT[1],anchor_prt)    # Anchor 2
    initialEst[56,:] = prt_point(anchor_GT[2],anchor_prt)    # Anchor 3
    initialEst[57,:] = prt_point(anchor_GT[3],anchor_prt)    # Anchor 4
    
    initialEst_pd = pd.DataFrame(initialEst)
    initialEst_pd.to_csv('../dataset/estimates_7.csv', header=False, index=False)
    values = np.concatenate((np.array((initialEst[49,0], initialEst[50,0], initialEst[51,0], initialEst[52,0], initialEst[53,0])),
                             initialEst[54], initialEst[55], initialEst[56], initialEst[57], list(map(float, run_cpp_program()))))
    result.append(values)

result = np.array(result)
result_pd = pd.DataFrame(result)
result_pd.columns = ['s0', 'm00', 'n00', 'k00', 'h00', 'A10_x', 'A10_y', 'A20_x', 'A20_y', 'A30_x', 'A30_y', 'A40_x', 'A40_y',
                     's', 'm0', 'n0', 'k0', 'h0', 'A1_x', 'A1_y', 'A2_x', 'A2_y', 'A3_x', 'A3_y', 'A4_x', 'A4_y', 'X0_x', 'X0_y']
# result_pd.to_csv(file_path, mode='a', index=False, header=not file_exists)

In [161]:
rmse_m0 = np.sqrt(np.mean((result_pd['m0'] - cable_GT[0]) ** 2))
rmse_n0 = np.sqrt(np.mean((result_pd['n0'] - cable_GT[1]) ** 2))
rmse_k0 = np.sqrt(np.mean((result_pd['k0'] - cable_GT[2]) ** 2))
rmse_h0 = np.sqrt(np.mean((result_pd['h0'] - cable_GT[3]) ** 2))

In [162]:
plt.subplots()
%matplotlib qt

plt.scatter(result_pd['A10_x'], result_pd['A10_y'], color="tab:cyan", label="Anchor 1 Initial Values")
plt.scatter(result_pd['A20_x'], result_pd['A20_y'], color="tab:pink", label="Anchor 2 Initial Values")
plt.scatter(result_pd['A30_x'], result_pd['A30_y'], color="gold", label="Anchor 3 Initial Values")
plt.scatter(result_pd['A40_x'], result_pd['A40_y'], color="limegreen", label="Anchor 4 Initial Values")
plt.scatter(result_pd['A1_x'] - result_pd['A3_x'], result_pd['A1_y'] - result_pd['A3_y'], label="Anchor 1 Optimization Result", color="tab:blue")
plt.scatter(result_pd['A2_x'] - result_pd['A3_x'], result_pd['A2_y'] - result_pd['A3_y'], label="Anchor 2 Optimization Result", color="tab:purple")
plt.scatter(result_pd['A3_x'] - result_pd['A3_x'], result_pd['A3_y'] - result_pd['A3_y'], label="Anchor 3 Optimization Result", color="tab:orange")
plt.scatter(result_pd['A4_x'] - result_pd['A3_x'], result_pd['A4_y'] - result_pd['A3_y'], label="Anchor 4 Optimization Result", color="green")
plt.scatter(result_pd['X0_x'], result_pd['X0_y'], label="X0", color="green")

plt.legend()

plt.subplots()

plt.subplot(4,1,1)
plt.hist(result_pd['m0'], bins=100, edgecolor='black')
mean = np.mean(result_pd['m0'])
std_dev = np.std(result_pd['m0'])
plt.legend([f'Without Perturbation: {cable_GT[0]}\nMean: {mean:.3f}\nRMSE: {rmse_m0:.3f}'])
plt.ylabel('$m_0$')

plt.subplot(4,1,2)
plt.hist(result_pd['n0'], bins=100, edgecolor='black')
mean = np.mean(result_pd['n0'])
std_dev = np.std(result_pd['n0'])
plt.legend([f'Without Perturbation: {cable_GT[1]}\nMean: {mean:.3f}\nRMSE: {rmse_n0:.3f}'])
plt.ylabel('$n_0$')

plt.subplot(4,1,3)
plt.hist(result_pd['k0'], bins=100, edgecolor='black')
mean = np.mean(result_pd['k0'])
std_dev = np.std(result_pd['k0'])
plt.legend([f'Without Perturbation: {cable_GT[2]}\nMean: {mean:.3f}\nRMSE: {rmse_k0:.3f}'])
plt.ylabel('$k_0$')

plt.subplot(4,1,4)
plt.hist(result_pd['h0'], bins=100, edgecolor='black')
mean = np.mean(result_pd['h0'])
std_dev = np.std(result_pd['h0'])
plt.legend([f'Without Perturbation: {cable_GT[3]}\nMean: {mean:.3f}\nRMSE: {rmse_h0:.3f}'])
plt.ylabel('$h_0$')
plt.xlabel('Length (m)')
plt.suptitle(f'$Perturbation={cable_prt}$')

mplcursors.cursor(multiple=True)
plt.show()